## Comparing two bus systems using GTFS data in Python

Daniel Ritter | *GEOG 6180 Geoprocessing in Python* | Final Project

Transportation agencies share transit data using the [General Transit Feed Specification](https://gtfs.org/). GTFS is a text-based standard data format used by over 10,000 agencies in over 100 countries. The following script uses the [gtfs_functions](https://pypi.org/project/gtfs-functions/) Python package to describe and compare two bus systems.

In [1]:
# pip install gtfs_functions==2.0.8

import gtfs_functions as gf
import geopandas as gpd
import pandas as pd
import keplergl as kp
import tabulate as tab

### Read GTFS data feed






`Feed()` creates an object of class `Feed` from a zipped GTFS feed, which is then used to create individual dataframes and geodataframes based on the text files within the feed (modified to fit the `gtfs_functions` workflow).

In [2]:
# Define file paths for GTFS zip files
gtfs_pathA = r"C:\Users\danie\OneDrive\Documents\Projects\GEOG6180\Project\GTFS-Feeds\GTFS-UTA.zip"
gtfs_pathB = r"C:\Users\danie\OneDrive\Documents\Projects\GEOG6180\Project\GTFS-Feeds\GTFS-VRT.zip"

# Define time windows (24-hour clock)
window = [0, 6, 9, 15, 19, 22, 24]

# Read feed
feedA = gf.Feed(gtfs_pathA, time_windows = window)
feedB = gf.Feed(gtfs_pathB, time_windows = window)

# Create objects from files for system A
agencyA = feedA.agency
routesA = feedA.routes
tripsA = feedA.trips
stopsA = feedA.stops
stoptimesA = feedA.stop_times
shapesA = feedA.shapes

# Create objects from files for system B
agencyB = feedB.agency
routesB = feedB.routes
tripsB = feedB.trips
stopsB = feedB.stops
stoptimesB = feedB.stop_times
shapesB = feedB.shapes

# Identify agencies
agencyA_name = agencyA["agency_name"][0]
agencyB_name = agencyB["agency_name"][0]

# Define function to filter for bus routes
def busfilter(routes): 
    busonly = routes[routes["route_type"] == 3]
    return busonly

INFO:root:Reading "agency.txt".
INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-01-16'] with 5444 trips.
INFO:root:In that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-01-16.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".
INFO:root:Reading "agency.txt".
INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2023-09-04', '2023-02-16', '2023-03-23', '2023-03-24', '2024-05-28', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-

INFO:root:In that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2023-09-04.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".


`Feed()` imports routes.txt as a regular dataframe without a geometry attribute. Route geometries are stored in the *shapes* object and linked to the *routes* dataframe through the *trip* dataframe (which shares a *shape_id* attribute with *shapes* and a *route_id* attribute with *routes*). The `lines_freq` and `stops_freq` functions makes this linkage and generate trip counts based on the time windows defined earlier.

In [3]:
# Calculate stop and route frequencies
stop_freqA = feedA.stops_freq
stop_freqB = feedB.stops_freq
line_freqA = feedA.lines_freq
line_freqB = feedB.lines_freq

# Assign route mode based on route_id
line_freq_modeA = line_freqA.merge(routesA[["route_id", "route_type"]], on = "route_id")
line_freq_modeB = line_freqB.merge(routesB[["route_id", "route_type"]], on = "route_id")

# Filter for bus only routes
line_freq_busA = busfilter(line_freq_modeA)
line_freq_busB = busfilter(line_freq_modeB)

# Create route object with geometry
busroutes_geomA = line_freq_busA.drop_duplicates(subset = "route_id")
busroutes_geomB = line_freq_busB.drop_duplicates(subset = "route_id")

### Calculate descriptive statistics

In order for route length to be meaningful, the CRS must be transformed from WGS 84 (degrees). US National Atlas Equal Area (EPSG 2163) is optimized for the United States, which makes it a reasonable default choice. 

In [4]:
# Reproject to EPSG 2163 (National Atlas Equal Area)
busroutes_geomA = busroutes_geomA.to_crs(2163)
busroutes_geomB = busroutes_geomB.to_crs(2163)

# Calculate route length and convert to miles
busroutes_lengthA = busroutes_geomA.length.divide(1609.34)
busroutes_lengthB = busroutes_geomB.length.divide(1609.34)

# Find min, max, mean, and total route lengths
route_meanA = busroutes_lengthA.mean().round(1)
route_meanB = busroutes_lengthB.mean().round(1)
net_lenA = busroutes_lengthA.sum().round(1)
net_lenB = busroutes_lengthB.sum().round(1)

# Summarize daily trips by route
route_tripsA = line_freq_busA.groupby("route_name")["ntrips"].sum()
route_tripsB = line_freq_busB.groupby("route_name")["ntrips"].sum()

# Find route with the most trips
rtrip_totalA = line_freq_busA["ntrips"].sum()
rtrip_totalB = line_freq_busB["ntrips"].sum()
rtrip_maxA = route_tripsA.max()
rtrip_maxB = route_tripsB.max()
rtrip_maxIDa = route_tripsA.idxmax()
rtrip_maxIDb = route_tripsB.idxmax()

The `print` statements use a mix of defined variables and functions applied to dataframes. `if-elif-else` statements are used for direct comparisons between the two systems. An `if-else` statement would be adequate for almost every case, but there is a very slight chance that two systems could have the same value.

In [5]:
print("System Comparison: {} vs. {}\n".format(agencyA_name, agencyB_name))

print("{} has {} bus routes, while {} has {} bus routes.\n".format(agencyA_name, len(busroutes_geomA), 
                                                                   agencyB_name, len(busroutes_geomB)))

print("The average route length in the {} system is {} miles.".format(agencyA_name, route_meanA))
print("The average route length in the {} system is {} miles.".format(agencyB_name, route_meanB))

System Comparison: Utah Transit Authority vs. Valley Regional Transit

Utah Transit Authority has 81 bus routes, while Valley Regional Transit has 22 bus routes.

The average route length in the Utah Transit Authority system is 14.4 miles.
The average route length in the Valley Regional Transit system is 10.6 miles.


In [6]:
# Print length statement based on which system is longer
if net_lenA > net_lenB: 
    print("{}'s system is {}% longer than {}'s system ({} miles vs. {} miles).\n".format(agencyA_name, 
                                                                                         ((net_lenA - net_lenB) / net_lenB * 100).round(1), 
                                                                                         agencyB_name, net_lenA, net_lenB))
elif net_lenB > net_lenA: 
    print("{}'s system is {}% longer than {}'s system ({} miles vs. {} miles).\n".format(agencyB_name, 
                                                                                         ((net_lenB - net_lenA) / net_lenA * 100).round(1), 
                                                                                         agencyA_name, net_lenB, net_lenA))
else: 
    print("Both systems are the same length ({} miles).\n".format(net_lenA))


# Print stop statement based on which system has more stops
if len(stopsA) > len(stopsB): 
    print("{} has {}% more stops than {} ({} stops vs. {} stops).\n".format(agencyA_name, 
                                                                            round(((len(stopsA) - len(stopsB)) / len(stopsB) * 100), 1), 
                                                                            agencyB_name, len(stopsA), len(stopsB)))
elif len(stopsB) > len(stopsA): 
    print("{} has {}% more stops than {} ({} stops vs. {} stops).\n".format(agencyB_name, 
                                                                            round(((len(stopsB) - len(stopsA)) / len(stopsB) * 100), 1), 
                                                                            agencyA_name, len(stopsB), len(stopsA)))
else: 
    print("Both transit agencies have the same number of stops ({} stops).\n".format(len(stopsA)))


# Print trip statement based on which system has more trips
if rtrip_totalA > rtrip_totalB:  
    print("{} offers {}% more daily trips than {} ({} trips vs. {} trips).\n".format(agencyA_name, 
                                                                                     round(((rtrip_totalA - rtrip_totalB) / rtrip_totalB * 100), 1), 
                                                                                     agencyB_name, rtrip_totalA, rtrip_totalB))

elif rtrip_totalB > rtrip_totalA: 
    print("{} offers {}% more daily trips than {} ({} trips vs. {} trips).\n".format(agencyB_name, 
                                                                                     round(((rtrip_totalB - rtrip_totalA) / rtrip_totalA * 100), 1), 
                                                                                     agencyA_name, rtrip_totalB, rtrip_totalA))
else: 
    print("Both transit agencies have the same number of daily trips on their busiest day ({} trips).\n".format(rtrip_totalA))

# Print route details 
print("The {} route with the most daily trips (all modes) is {} ({} trips).".format(agencyA_name, rtrip_maxIDa, rtrip_maxA))
print("The {} route with the most daily trips (all modes) is {} ({} trips).".format(agencyB_name, rtrip_maxIDb, rtrip_maxB))

Utah Transit Authority's system is 399.7% longer than Valley Regional Transit's system (1169.4 miles vs. 234.0 miles).

Utah Transit Authority has 534.0% more stops than Valley Regional Transit (5243 stops vs. 827 stops).

Utah Transit Authority offers 591.7% more daily trips than Valley Regional Transit (4773 trips vs. 690 trips).

The Utah Transit Authority route with the most daily trips (all modes) is 830X UTAH VALLEY EXPRESS (298 trips).
The Valley Regional Transit route with the most daily trips (all modes) is 9 State Street (99 trips).


### Visualize system

`gtfs_functions` contains a basic mapping function built on `folium`, but a mapping-specific Python package like `kepler.gl` is needed for more intensive mapping.

In [7]:
## Create basic map of system A
# routemapA = kp.KeplerGl(height = 600, data = {"routesA": route_geomA}, show_docs = False)
# routemapA

Once the default visualization is modified within the widget, the configuration can be saved within Jupyter and reloaded to maintain the modifications when rerunning the script.

In [8]:
## Modify visualization and save configuration for system A
# with open("routemapA_config.py", "w") as f: 
#   f.write("config = {}".format(routemapA.config))

# Load preset config
%run routemapA_config.py

# Create basic system map with preset config
kp.KeplerGl(height = 600, data = {"routesA": busroutes_geomA}, config = config, show_docs = False)

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '287bso', 'type': '…

![Screenshot of system A map](https://raw.githubusercontent.com/dritter4/GEOG-6180/main/Compare-Systems-GTFS/RouteMapA-Screenshot.png)

In [9]:
## Create basic map of system B
# routemapB = kp.KeplerGl(height = 600, data = {"routesB": route_geomB}, show_docs = False)
# routemapB

In [10]:
## Modify visualization and save configuration for system B
# with open("routemapB_config.py", "w") as f: 
#   f.write("config = {}".format(routemapB.config))

# Load preset config
%run routemapB_config.py

# Create basic system map with preset config
kp.KeplerGl(height = 600, data = {"routesB": busroutes_geomB}, config = config, show_docs = False)

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'lzv9pj', 'type': '…

![Screenshot of system B map](https://raw.githubusercontent.com/dritter4/GEOG-6180/main/Compare-Systems-GTFS/RouteMapB-Screenshot.png)

### Describe service frequency

In this script, frequency is calculated from 3:00 p.m. to 7:00 a.m., which represents bus service for the afternoon/evening commute. This time period can be changed by adjusting the time windows used when reading the GTFS feed and/or updating the parameters. Since frequency refers to how often a bus comes in a specific direction, either outbound or inbound should be defined. Both directions will generally have the same frequency, but there will be some differences depending on how buses are routed within a specific system.

A simple `for` loop is used to calculate the percentage of routes with each frequency and create a basic table, which is printed cleanly using the `tabulate` package.

In [11]:
# Define desired time period and direction
freq_window = "15:00-19:00"
freq_direction = 0

# Filter for time period and direction
line_freq_filterA = line_freq_busA.query(f'window == "{freq_window}" and direction_id == {freq_direction}')
line_freq_filterB = line_freq_busB.query(f'window == "{freq_window}" and direction_id == {freq_direction}')

# Define frequencies
bins = [0, 10, 20, 30, 40, 60, 1440]
binvals = ["0-10 minutes", "11-20 minutes", "21-30 minutes", "31-40 minutes", "41-60 minutes", "60+ minutes"]
binlength = len(binvals)

# Count number of routes for each frequency
bincountA = pd.cut(line_freq_filterA['min_per_trip'], bins = bins).value_counts(sort = False)
bincountB = pd.cut(line_freq_filterB['min_per_trip'], bins = bins).value_counts(sort = False)
totalroutesA = len(line_freq_filterA)
totalroutesB = len(line_freq_filterB)

# Create empty list
freqtable = []

# Populate list with percentage of routes in each frequency interval
for i in range(binlength): 
    freq = binvals[i]
    perA = round(bincountA.iloc[i] / totalroutesA * 100, 1) # Calculate percentage
    perB = round(bincountB.iloc[i] / totalroutesB * 100, 1) # Calculate percentage
    perA_str = f"{perA}%" # Add percentage symbol
    perB_str = f"{perB}%" # Add percentage symbol
    row = [freq, perA_str, perB_str]
    freqtable.append(row)

# Print frequency table
print("Percentage of routes in each system that operate with a specific frequency:\n")
print(tab.tabulate(freqtable, headers = ["Frequency", agencyA_name, agencyB_name]))

Percentage of routes in each system that operate with a specific frequency:

Frequency      Utah Transit Authority    Valley Regional Transit
-------------  ------------------------  -------------------------
0-10 minutes   2.5%                      0.0%
11-20 minutes  16.5%                     4.8%
21-30 minutes  29.1%                     23.8%
31-40 minutes  10.1%                     19.0%
41-60 minutes  27.8%                     38.1%
60+ minutes    13.9%                     14.3%


The visualizations options in `kepler.gl` are relatively limited compared to a dedicated GIS software like ArcGIS Pro, so route frequencies are binned into quantiles instead of the predefined ranges used above.

In [12]:
## Create map of route frequencies
# routefreqmapA = kp.KeplerGl(height = 600, data = {"routefreqA": line_freq_filterA}, show_docs = False)
# routefreqmapA

In [13]:
## Modify visualization and save configuration
# with open("routefreqmapA_config.py", "w") as f: 
#   f.write("config = {}".format(routefreqmapA.config))

# Load preset config
%run routefreqmapA_config.py

# Create basic system map with preset config
kp.KeplerGl(height = 600, data = {"routefreqA": line_freq_filterA}, config = config, show_docs = False)

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'z0wzn0o', 'type': …

![Screenshot of route frequency A map](https://raw.githubusercontent.com/dritter4/GEOG-6180/main/Compare-Systems-GTFS/RouteFrequencyA-Screenshot.png)

In [14]:
## Create map of route frequencies
# routefreqmapB = kp.KeplerGl(height = 600, data = {"routefreqB": line_freq_filterB}, show_docs = False)
# routefreqmapB

In [15]:
## Modify visualization and save configuration
# with open("routefreqmapB_config.py", "w") as f: 
#  f.write("config = {}".format(routefreqmapB.config))

# Load preset config
%run routefreqmapB_config.py

# Create basic system map with preset config
kp.KeplerGl(height = 600, data = {"routefreqB": line_freq_filterB}, config = config, show_docs = False)

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'qfyu7g9', 'type': …

![Screenshot of route frequency B map](https://raw.githubusercontent.com/dritter4/GEOG-6180/main/Compare-Systems-GTFS/RouteFrequencyB-Screenshot.png)